# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np
import click

In [ ]:
#| export
from infepy.rbf import RBF

In [ ]:
#| export
from infepy.preprocessing import read_landmarks, read_nodes, _check_landmarks, write_output
from infepy.utils import read_toml, multiple_targets, _merge_path, to_ls_dyna, read_csv_file,from_df_to_np

In [ ]:
#| export
def morphing(source_landmarks: np.ndarray,  # Landmarks of the source mesh
             target_landmarks: np.ndarray,  # Target Landmarks
             source_mesh: np.ndarray, # Coordinates of the mesh/ mesh 
             ):
    "Morph the target mesh with RBF function - Thin Plate Spine."
    
    rbf = RBF(original_control_points=from_df_to_np(source_landmarks) , 
              deformed_control_points= from_df_to_np(target_landmarks),
              func='thin_plate_spline', radius=1.0)

    return rbf(from_df_to_np(source_mesh))

In [ ]:
#| export
# @click.command()
# @click.option('--name', prompt='Filename source mesh', help=' Template mesh name')
# @click.option('--file', prompt = 'Extension source mesh', help=' Source mesh type key or csv.')
# @click.option('--n_target', default = 1 ,prompt = 'Number of target', help='Number of targets.')

def do_morphing():
    config = read_toml()
    source_landmarks = read_landmarks(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
    template_mesh = read_nodes(_merge_path(config['source']['path'],config['source']['filename_mesh']))
    
    if multiple_targets() == False: # single target
        target_landmarks = read_landmarks(_merge_path(config['target']['path'],config['target']['filename_landmarks']))
        _check_landmarks(source_landmarks, target_landmarks)
        morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh)
        write_output(morphed_mesh,
                     _merge_path(config['target']['path'], 'morphed.key'),
                     _merge_path(config['source']['path'],config['source']['filename_mesh']))
    else:
        targets_folder = multiple_targets()
        for folder in targets_folder:
            folder_path = os.path.join(config['target']['path'],folder)
            target_landmarks = read_landmarks(_merge_path(folder_path,config['target']['filename_landmarks']))
            _check_landmarks(source_landmarks, target_landmarks)
            morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh)
            write_output(morphed_mesh,
                      _merge_path(folder_path,'morphed.key'),
                      _merge_path(config['source']['path'],config['source']['filename_mesh']))
    return

In [ ]:
#| export
if __name__ == "__main__":
    do_morphing()

SyntaxError: invalid syntax (1641638307.py, line 4)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()